In [1]:
from pyspark.sql import functions as F
from service.utils.spark import get_spark_session
spark = get_spark_session(dev=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 05:47:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
test_namespace = 'gold' 
# spark.sql(f"DROP NAMESPACE IF EXISTS {test_namespace} CASCADE")
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {test_namespace}").show()

++
||
++
++



In [3]:
product_df = spark.read.csv("s3a://warehousedev/bronze/tsv/product.tsv", header=True, sep='\t')
essential_product_df = product_df.select('product_id', 'category')
essential_product_df.show(2)

25/09/30 05:47:35 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+--------------------+
|          product_id|            category|
+--------------------+--------------------+
|18ea29d5e42eea2c7...|agro_industry_and...|
|a79e27ad902bfc673...|agro_industry_and...|
+--------------------+--------------------+
only showing top 2 rows



In [4]:
order_item_df = spark.read.csv("s3a://warehousedev/bronze/tsv/order_item.tsv", header=True, sep='\t')
order_product_category = essential_product_df.join(order_item_df, on='product_id', how='inner')
order_product_category = order_product_category.sort('product_id')
order_product_category.show(2)

+--------------------+---------+--------------------+-------------+--------------------+-------------------+------+-------------+
|          product_id| category|            order_id|order_item_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+---------+--------------------+-------------+--------------------+-------------------+------+-------------+
|00066f42aeeb9f300...|perfumery|f30149f4a8882a088...|            1|5670f4db5b62c43d5...|2018-05-24 18:58:59|101.65|        18.59|
|00088930e925c41fd...|     auto|f5eda0ded77c1293b...|            1|7142540dd4c91e223...|2017-12-18 19:32:19| 129.9|        13.93|
+--------------------+---------+--------------------+-------------+--------------------+-------------------+------+-------------+
only showing top 2 rows



In [5]:
# 하나의 order에 여러 item이 있을 수 있으므로, order_item_id는 남겨둔다.
# `total_price` 는 상품가와 배송비를 합산한 가격
order_product = order_product_category.withColumn('total_price', F.round(F.col('price') + F.col('freight_value'), 4))
order_product = order_product.drop('price', 'freight_value')
order_product.show(n=5)

+--------------------+--------------+--------------------+-------------+--------------------+-------------------+-----------+
|          product_id|      category|            order_id|order_item_id|           seller_id|shipping_limit_date|total_price|
+--------------------+--------------+--------------------+-------------+--------------------+-------------------+-----------+
|00066f42aeeb9f300...|     perfumery|f30149f4a8882a088...|            1|5670f4db5b62c43d5...|2018-05-24 18:58:59|     120.24|
|00088930e925c41fd...|          auto|f5eda0ded77c1293b...|            1|7142540dd4c91e223...|2017-12-18 19:32:19|     143.83|
|0009406fd7479715e...|bed_bath_table|0bf736fd0fd5169d6...|            1|4a3ca9315b744ce9f...|2017-12-29 16:12:36|      242.1|
|000b8f95fcb9e0096...|    housewares|3aba44d8e554ab4bb...|            1|40ec8ab6cdafbcc4f...|2018-08-16 13:35:21|       78.5|
|000b8f95fcb9e0096...|    housewares|6f0dfb5b5398b271c...|            1|40ec8ab6cdafbcc4f...|2018-08-07 22:15:19|     

In [6]:
order_product.count()

112650

In [9]:
# tmp_order_timestamp = spark.read.table(f'{test_namespace}.tmp_order_timestamp')
tmp_order_timestamp = spark.read.table(f'silver.order_timestamp')
delivered_order_id = tmp_order_timestamp.select('order_id').distinct()

In [10]:
tmp_order_product = order_product.join(delivered_order_id, on='order_id', how='inner')

In [11]:
tmp_order_product.show(2)

+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
|            order_id|          product_id|  category|order_item_id|           seller_id|shipping_limit_date|total_price|
+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
|00010242fe8c5a6d1...|4244733e06e7ecb49...|cool_stuff|            1|48436dade18ac8b2b...|2017-09-19 09:45:35|      72.19|
|00018f77f2f0320c5...|e5f2d52b802189ee6...|  pet_shop|            1|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|     259.83|
+--------------------+--------------------+----------+-------------+--------------------+-------------------+-----------+
only showing top 2 rows



In [12]:
# tmp_order_product.writeTo(f"{test_namespace}.tmp_order_product").createOrReplace()
tmp_order_product.writeTo(f"silver.order_product").createOrReplace()

In [13]:
spark.stop()